# Main

## Input

### Import libraries

In [ ]:
import naas_data_product
import naas_python
import os
import sys
import pydash as _

# Make it possible to do "import abi"
sys.path.append(naas_data_product.UTILS_PATH)

from abi.storagemanager import StorageManager
from abi.assetmanager import AssetManager

# This should be moved into a class instead of import *
from abi.workspacemanager import get_storage_credentials
from abi.utils import open_yaml_file

### Setup variables

In [ ]:
config_path = os.path.join(naas_data_product.ROOT_PATH, 'config.yml')
config_secret_path = os.path.join(naas_data_product.ROOT_PATH, 'config.secret.yml')
api_key = naas_python.utils.domains_base.authorization.NaasSpaceAuthenticatorAdapter().jwt_token()

## Model

### Get config variables

In [ ]:
workspace_id = None
storage_name = "abi"
access_key_id = None
secret_access_key = None
session_token = None
bucket_name = None
bucket_prefix = None

# Get data from YAML file
config_data = open_yaml_file(config_path) | open_yaml_file(config_secret_path)
if config_data:
    workspace_id = _.get(config_data, "config.workspace_id")
    storage_name = _.get(config_data, "config.storage_name")
    access_key_id = _.get(config_data, "storage.access_key_id")
    secret_access_key = _.get(config_data, "storage.secret_access_key")
    session_token = _.get(config_data, "storage.session_token")
    bucket_name = _.get(config_data, "storage.bucket_name")
    bucket_prefix = _.get(config_data, "storage.bucket_prefix")

### Get credentials

In [ ]:
# if not access_key_id and not secret_access_key:
credentials, workspace_id = get_storage_credentials(workspace_id=workspace_id, storage_name=storage_name)
if len(credentials) > 0:
    access_key_id = _.get(credentials, "credentials.s3.access_key_id")
    secret_access_key = _.get(credentials, "credentials.s3.secret_key")
    session_token = _.get(credentials, "credentials.s3.session_token")
    endpoint_url = _.get(credentials, "credentials.s3.endpoint_url")
    bucket_name = endpoint_url.split("s3://")[1].split("/")[0]
    bucket_prefix = endpoint_url.split(f"{bucket_name}/")[1]
    region_name = _.get(credentials, "credentials.s3.region_name")

## Output

### Storage Manager

In [ ]:
sm = StorageManager(access_key_id, secret_access_key, session_token, bucket_name, bucket_prefix)

### Asset Manager

In [ ]:
am = AssetManager(workspace_id, storage_name, sm, api_key)